from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Resource can be required for some backends, e.g. Jaeger
# If resource wouldn't be set - traces wouldn't appears in Jaeger
resource = Resource(attributes={
    "service.name": "service"
})

trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)

otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)

span_processor = BatchSpanProcessor(otlp_exporter)

trace.get_tracer_provider().add_span_processor(span_processor)

with tracer.start_as_current_span("foo"):
    print("Hello world!")

# Working with Spans

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [2]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [3]:
from trulens_eval import Tru
Tru().reset_database()
Tru().start_dashboard(_dev=repo, force=True)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.43.124:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [4]:
# Create custom app:
custom_app = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
tru_custom_app = TruCustomApp(
    custom_app,
    app_id="customapp",
)

custom_agent = custom_app.agents[0]

tru_custom_agent = TruCustomApp(
     custom_agent,
     app_id="customapp_agent"
)

In [5]:
custom_app.respond_to_query("testing")

'The answer to testing is probably Generating from \'rELEVANT CHUNK: testing,<table>\\n\\n            <tr>\\n                <td>examples.expositional.end2end_apps.custom_app.custom_tool</td>\\n                <td>invoke</td>\\n                <td><pre><code> 74           0 RESUME                   0\\n\\n 76           2 LOAD_GLOBAL              1 (NULL + list)\\n             12 LOAD_GLOBAL              3 (NULL + superstack)\\n             22 CALL                     0\\n             30 CALL                     1\\n             38 LOAD_GLOBAL              4 (CustomStackTool)\\n             48 STORE_ATTR               3 (last_stack)\\n\\n 78          58 LOAD_CONST               1 (\\\'<table>\\\\n\\\')\\n             60 STORE_FAST               2 (ret)\\n\\n 79          62 LOAD_GLOBAL              4 (CustomStackTool)\\n             72 LOAD_ATTR                6 (last_stack)\\n             92 GET_ITER\\n        >>   94 FOR_ITER               155 (to 408)\\n             98 STORE_FAST     

In [6]:
instruments.Instrument().print_instrumentation()

Module examples.expositional.end2end_apps.custom_app.custom_agent*
  Class examples.expositional.end2end_apps.custom_app.custom_agent.CustomAgent
    Method invoke: (self, data: str) -> str
      Span type: SpanType.AGENT (static), spanner: <function CustomAgent.set_agent_span at 0x17bf80ae0>
      Span type: SpanType.AGENT (0x17c322ab0), spanner: <function CustomAgent.set_agent_span at 0x17bf80ae0>
      Span type: SpanType.AGENT (0x17c287110), spanner: <function CustomAgent.set_agent_span at 0x17bf80ae0>

Module examples.expositional.end2end_apps.custom_app.custom_app*
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomApp
    Method process_chunk_by_random_tool: (self, chunk_and_score: Tuple[str, float]) -> str
    Method get_context: (self, input: str)
    Method respond_to_query: (self, query: str)
    Method arespond_to_query: (self, input: str)
  Class examples.expositional.end2end_apps.custom_app.custom_app.CustomTemplate
    Method fill: (self, question, an

In [7]:
tru_custom_app.print_instrumented()

Components:
	TruCustomApp (Other) at 0x17c3a4af0 with path __app__
	CustomApp (Custom) at 0x17c322e40 with path __app__.app
	CustomAgent (Custom) at 0x17c322ab0 with path __app__.app.agents[0]
	CustomApp (Custom) at 0x17bfe1370 with path __app__.app.agents[0].app
	CustomLLM (Custom) at 0x17c354560 with path __app__.app.agents[0].app.llm
	CustomMemory (Custom) at 0x1069aa7e0 with path __app__.app.agents[0].app.memory
	CustomReranker (Custom) at 0x17c3541a0 with path __app__.app.agents[0].app.reranker
	CustomRetriever (Custom) at 0x1069a9550 with path __app__.app.agents[0].app.retriever
	CustomTemplate (Custom) at 0x17c354590 with path __app__.app.agents[0].app.template
	CustomTool (Custom) at 0x17c3542f0 with path __app__.app.agents[0].app.tools[0]
	CustomTool (Custom) at 0x17c3542c0 with path __app__.app.agents[0].app.tools[1]
	CustomTool (Custom) at 0x17c354230 with path __app__.app.agents[0].app.tools[2]
	CustomStackTool (Custom) at 0x17c354320 with path __app__.app.agents[0].app.too

In [8]:
tru_custom_agent.print_instrumented()

Components:
	TruCustomApp (Other) at 0x175dd2e90 with path __app__
	CustomAgent (Custom) at 0x17c322ab0 with path __app__.app
	CustomApp (Custom) at 0x17bfe1370 with path __app__.app.app
	CustomLLM (Custom) at 0x17c354560 with path __app__.app.app.llm
	CustomMemory (Custom) at 0x1069aa7e0 with path __app__.app.app.memory
	CustomReranker (Custom) at 0x17c3541a0 with path __app__.app.app.reranker
	CustomRetriever (Custom) at 0x1069a9550 with path __app__.app.app.retriever
	CustomTemplate (Custom) at 0x17c354590 with path __app__.app.app.template
	CustomTool (Custom) at 0x17c3542f0 with path __app__.app.app.tools[0]
	CustomTool (Custom) at 0x17c3542c0 with path __app__.app.app.tools[1]
	CustomTool (Custom) at 0x17c354230 with path __app__.app.app.tools[2]
	CustomStackTool (Custom) at 0x17c354320 with path __app__.app.app.tools[3]

Methods:
Object at 0x1069aa7e0:
	<function CustomMemory.remember at 0x17bf81440> with path __app__.app.app.memory
Object at 0x1069a9550:
	<function CustomRetrie

In [15]:
# Create custom app:
custom_app1 = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
tru_custom_app1 = TruCustomApp(
    custom_app1,
    app_id="customapp1",
)

# Create another custom app:
custom_app2 = CustomApp(delay=0.0, alloc=0)

# Create another trulens wrapper:
tru_custom_app2 = TruCustomApp(
    custom_app2,
    app_id="customapp2",
)

# Sequential contexts.
with tru_custom_app1 as recorder_app1:
    res_app1 = custom_app1.respond_to_query("hello 1")

with tru_custom_app2 as recorder_app2:
    res_app2 = custom_app2.respond_to_query("hello 2")

# Nested sequential contexts.
with tru_custom_app1 as recorder_app1:
    res_app1 = custom_app1.respond_to_query("hello 1")

    with tru_custom_app2 as recorder_app2:
        res_app2 = custom_app2.respond_to_query("hello 2")

In [13]:
res_app1

"The answer to hello 1 is probably Generating from 'Relevant chunk: HELLO 1,Relevant chunk: 1 olleh1 olleh1 olleh' and 56 bytes with temperature 0.5 or something ..."

In [14]:
res_app2

'The answer to hello 2 is probably Generating from "Relevant chunk: hello 2,\'Relevant chunk: 2 olleh2 olleh2 olleh\'" and 56 bytes with temperature 0.5 or something ...'

In [4]:
# Create custom app:
custom_app = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
tru_custom_app = TruCustomApp(
    custom_app,
    app_id="customapp",
)

# A component of the custom app:
custom_agent = custom_app.agents[0]

# A recorder of the component:
tru_custom_agent = TruCustomApp(
     custom_agent,
     app_id="customapp_agent"
)

with tru_custom_app as recorder_app:
    res_app = custom_app.respond_to_query(f"hello")

    with tru_custom_agent as recorder_agent:
        res_agent = custom_agent.invoke("hello")

rec_app = recorder_app.get() # Everything

rec_agent = recorder_agent.get() # Just the agent

app.memory: instrumenting remember=<function CustomMemory.remember at 0x17cb85440>
app.retriever: instrumenting retrieve_chunks=<function CustomRetriever.retrieve_chunks at 0x17cb85b20>
app.llm: instrumenting generate=<function CustomLLM.generate at 0x17cb85120>
app.template: instrumenting fill=<function CustomTemplate.fill at 0x17cb849a0>
app.reranker: instrumenting rerank=<function CustomReranker.rerank at 0x17cb85760>
app: instrumenting process_chunk_by_random_tool=<function CustomApp.process_chunk_by_random_tool at 0x17cb86160>
app: instrumenting get_context=<function CustomApp.get_context at 0x17cb86200>
app: instrumenting respond_to_query=<function CustomApp.respond_to_query at 0x17cb862a0>
app: instrumenting arespond_to_query=<function CustomApp.arespond_to_query at 0x17cb86340>
app.tools[0]: instrumenting invoke=<function CustomTool.invoke at 0x17cb85da0>
app.tools[1]: instrumenting invoke=<function CustomTool.invoke at 0x17cf58400>
adding app ensamble agent 0 to apps for app.t

RuntimeError: App of type TruCustomApp no longer knows about object 0x17cdc0680 method invoke. Something might be going wrong.

In [ ]:
rec.calls[0].model_dump()

In [ ]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s).__name__,
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else -1,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)

In [ ]:
for span in spans:
    pprint(span)
    # pprint(span.model_dump())
    print()

In [ ]:
frozen = spans[1].freeze()

In [ ]:
frozen.attributes

In [ ]:
from trulens_eval.trace.span import Span

Span.thaw(frozen).model_dump()

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_tool import CustomStackTool
# setattr(CustomStackTool, "invoke", None)
getattr(CustomStackTool.invoke, "__tru_apps")

In [ ]:
dir(CustomStackTool.last_stack[0])

In [ ]:
import dis
for field in ["f_lasti", "f_lineno", "f_trace_lines", "f_trace_opcodes"]:#, "f_code", ]:
    print(CustomStackTool.last_stack[0].f_globals['__name__'])
    print(f"{field}: {getattr(CustomStackTool.last_stack[0], field)}")
    continue
    for cfield in [
        'co_argcount',
        'co_cellvars',
        'co_code',
        'co_consts',
        'co_exceptiontable',
        'co_filename',
        'co_firstlineno',
        'co_flags',
        'co_freevars',
        'co_kwonlyargcount',
        'co_lines',
        'co_linetable',
        'co_lnotab',
        'co_name',
        'co_names',
        'co_nlocals',
        'co_positions',
        'co_posonlyargcount',
        'co_qualname',
        'co_stacksize',
        'co_varnames'
    ]:
        if cfield == "co_code":
            print(f"  {cfield}: {dis.Bytecode(CustomStackTool.last_stack[0].f_code).dis()}")
        else:
            print(f"  {cfield}: {getattr(CustomStackTool.last_stack[0].f_code, cfield)}")

    

In [ ]:
b = dis.Bytecode(CustomStackTool.last_stack[0].f_code)

In [ ]:
import inspect
inspect.getmodule(CustomStackTool.last_stack[0].f_code).__name__

In [ ]:
for line in CustomStackTool.last_stack[0].f_code.co_lines():
    print(line)